In [1]:
import altair as alt
import numpy as np
import pandas as pd

In [2]:
today = pd.Timestamp.today()
tomorrow = today + pd.Timedelta(1, unit="D")

goal_mileage = 200
goal_start_date = pd.Timestamp("2021-10-11")
goal_end_date = pd.Timestamp("2021-12-17")

dates = pd.date_range(goal_start_date, goal_end_date)
scale_idx = (dates>=goal_start_date) & (dates<=min(today, goal_end_date))
mileage = np.linspace(0, goal_mileage, num=len(dates))

goal = pd.DataFrame({
    "Date": dates[scale_idx],
    "Mileage": mileage[scale_idx],
})

In [3]:
activities = pd.read_csv("Activities.csv")
activities["Distance"] = pd.to_numeric(activities["Distance"].str.replace(",", ""))
activities["Date"] = pd.to_datetime(activities["Date"])
activities = activities[
    (activities["Date"] >= goal_start_date) & (activities["Date"] <= goal_end_date)
]
activities = activities.sort_values(by=["Date"])
mask = activities["Activity Type"].str.lower().str.contains("swim")
activities.loc[mask, "Distance"] = activities.loc[mask, "Distance"] * 3 / 5280

In [4]:
progress = pd.DataFrame(activities["Date"].dt.floor("D"))
progress["Distance"] = activities["Distance"]
progress["Type"] = activities["Activity Type"]
progress = progress.groupby("Date").sum().cumsum().reset_index()

In [5]:
goal_line = alt.Chart(goal).mark_line(clip=True).encode(
    alt.X("monthdate(Date):O"),
    alt.Y("Mileage:Q"),
    color=alt.value("black"),
).properties(
    title="Mileage Tracker"
)

activity_bars = alt.Chart(activities).mark_bar().encode(
    alt.X("monthdate(Date):O"),
    alt.Y("sum(Distance):Q", title="Daily Mileage"),
    color="Activity Type",
)

progress_line = alt.Chart(progress).mark_line().encode(
    alt.X("monthdate(Date):O"),
    alt.Y("Distance:Q", title="Cumulative Mileage"),
    color=alt.value("green"),
)

alt.layer(
    activity_bars, goal_line + progress_line
).configure_axis(
    grid=True,
).properties(
    width=640,
    height=480,
)

alt.LayerChart(...)

In [6]:
goal_today = goal.loc[goal["Date"]<=today, "Mileage"].max()
progress_today = progress["Distance"].max()
difference = progress_today - goal_today
time_left = goal_end_date - today
ideal_daily_mileage = goal_mileage / (goal_end_date - goal_start_date).days
if difference < 0:
    daily_mileage = (-difference / time_left.days) + ideal_daily_mileage
elif progress_today < goal_mileage:
    daily_mileage = (goal_mileage - progress_today) / time_left.days
else:
    daily_mileage = 0

print(f"**As of {today.date()}**")
print(f"Progress: {progress_today:.1f} miles")
print(f"Goal: {goal_today:.1f} miles")
print(f"Difference: {difference:.1f} miles")
print(f"\nIdeal daily mileage: {ideal_daily_mileage:.1f} miles")
print(f"Time left: {time_left.days} days")
print(f"Daily mileage required to meet goal: {daily_mileage:.1f} miles")

**As of 2021-11-01**
Progress: 49.8 miles
Goal: 62.7 miles
Difference: -12.9 miles

Ideal daily mileage: 3.0 miles
Time left: 45 days
Daily mileage required to meet goal: 3.3 miles
